# VGG16_LargeFOV

## 1. Environment Setup
### 1.0. Check GPU

In [ ]:
! git clone https://github.com/chendonghp/deeplab-v1.git
%cd deeplab-v1/Implementation/
!pip install -U cython
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

from google.colab import drive
drive.mount('/content/drive')

! unzip /content/drive/MyDrive/data.zip -d .
! ls

In [3]:
import augmentation, model, train

import torch
import torchvision

# Ignore Warning
import warnings
warnings.filterwarnings(action='ignore')

from torch.utils.data import DataLoader
from data import LungImageDataset, colorDict

In [6]:
root = r"/mnt/d//GlassAI_data"

train_batch_size = 30
test_batch_size = 1

train_ratio = 0.9
size = 500
train_size = int(size*train_ratio)
train_range , val_range = (0,train_size), (train_size,size)


## 2. Data Preprocessing

In [7]:
train_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), # HWC to CHW
                                             augmentation.Resize((256, 256)), augmentation.RandomCrop((224, 224)), 
                                             augmentation.RandomHorizontalFlip(),
                                             augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

val_tf = augmentation.Mask_Aug(transforms=[augmentation.ToTensor(), augmentation.PILToTensor(), 
                                           augmentation.Resize((256, 256)),
                                           augmentation.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train = LungImageDataset(root,  size=train_range, transform=train_tf)
val = LungImageDataset(root, size=val_range, transform=val_tf)


train_loader = DataLoader(train, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)


# train_dataset = torchvision.datasets.SBDataset(root='./', image_set='train_noval', mode='segmentation', download=False, transforms=train_tf)
# train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4, pin_memory=True)

# val_dataset = torchvision.datasets.VOCSegmentation(root='./', year='2012', image_set='val', download=False, transforms=val_tf)
# val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=test_batch_size, shuffle=False, num_workers=4, pin_memory=True)

In [8]:
train_features, train_labels = next(iter(train_loader))

/mnt/d//GlassAI_data Images/mnt/d//GlassAI_data 226436371686740212350416577170247526575712.180r..png /mnt/d//GlassAI_data
Images  226436109251674021235044883349280112449112896.pngImages
 226436411756740212350418369188167840178848.90r..png/mnt/d//GlassAI_data
 Images 226436381696740212350417025174727571376160.270r..png


/mnt/d//GlassAI_data/mnt/d//GlassAI_data/mnt/d//GlassAI_data/mnt/d//GlassAI_data    ImagesImagesImagesImages    226436361686740212350416129165767526575712.png226436331866740212350414785152328332983776.png226436281776740212350412545129927929779744.png226436311756740212350413889143367840178848.180r..png



/mnt/d//GlassAI_data/mnt/d//GlassAI_data/mnt/d//GlassAI_data   Images/mnt/d//GlassAI_dataImagesImages    Images226436381716740212350417025174727660977056.90r..png226436105227674021235044704147488101697102144.png226436411766740212350418369188167884979296.png 


226436341796740212350415233156808019380640.png
/mnt/d//GlassAI_data/mnt/d//GlassAI_data/mnt/d//GlassAI_data  /mnt/d//GlassAI_data ImagesImages  ImagesImages 226436371696740212350416577170247571376160.270r..png  226436401766740212350417921183687884979296.90r..png
226436102228674021235044569746144102145102592.png
226436471646740212350421057215047347373920.90r..png

/mnt/d//GlassAI_data/mnt/d//GlassAI_data  ImagesImages  /mnt/d//Gla

In [10]:
train_labels.shape, train_features.shape

(torch.Size([30, 1, 224, 224]), torch.Size([30, 3, 224, 224]))

## 3. Train Model

In [8]:
num_classes = len(colorDict.keys())
init_weights = True

ignore_index = 255

gpu_id = 0
print_freq = 50
epoch_print = 10

save = True
epochs = 200

lr = 0.001
momentum = 0.9
weight_decay = 0.0005

In [9]:
vgg16_largefov =train.VGG16_LargeFOV(num_classes=num_classes, init_weights=init_weights, ignore_index=ignore_index, 
                                      gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

# VGG16_LargeFOV = train.VGG16_LargeFOV(num_classes=num_classes, init_weights=init_weights, ignore_index=ignore_index, 
#                                       gpu_id=gpu_id, print_freq=print_freq, epoch_print=epoch_print)

In [10]:
vgg16_largefov.train(train_loader, val_loader, save=save, epochs=epochs, lr=lr, momentum=momentum, weight_decay=weight_decay)
# VGG16_LargeFOV.train(train_loader, val_loader, save=save, epochs=epochs, lr=lr, momentum=momentum, weight_decay=weight_decay)

Epoch 1 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 50 - Train Loss : 1.725006, Test Loss : 1.419613, Test mIoU : 3.4907
Saved Best Model


*********************************** Best mIoU Updated ***********************************
Iteration : 100 - Train Loss : 1.334032, Test Loss : 1.229714, Test mIoU : 3.8550
Saved Best Model


*********************************** Best mIoU Updated ***********************************
Iteration : 150 - Train Loss : 1.105385, Test Loss : 1.019418, Test mIoU : 8.4972
Saved Best Model

Epoch 6 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 50 - Train Loss : 0.492005, Test Loss : 0.461120, Test mIoU : 45.0124
Saved Best Model


*********************************** Best mIoU Updated ***********************************
Iteration : 100 - Train Loss : 0.672748, Test Loss : 0.472144, Test mIoU : 45.6521
Saved Best Model


******


*********************************** Best mIoU Updated ***********************************
Iteration : 50 - Train Loss : 0.181136, Test Loss : 0.379694, Test mIoU : 59.5781
Saved Best Model

Iteration : 100 - Train Loss : 0.137960, Test Loss : 0.399642, Test mIoU : 58.6219
Iteration : 150 - Train Loss : 0.158613, Test Loss : 0.388408, Test mIoU : 59.4652
Epoch 106 Started...

*********************************** Best mIoU Updated ***********************************
Iteration : 50 - Train Loss : 0.148515, Test Loss : 0.380978, Test mIoU : 59.8859
Saved Best Model

Iteration : 100 - Train Loss : 0.184816, Test Loss : 0.384824, Test mIoU : 59.7185
Iteration : 150 - Train Loss : 0.165988, Test Loss : 0.395679, Test mIoU : 58.7030
Epoch 111 Started...
Iteration : 50 - Train Loss : 0.154349, Test Loss : 0.386386, Test mIoU : 59.6293
Iteration : 100 - Train Loss : 0.198688, Test Loss : 0.386850, Test mIoU : 59.8337

*********************************** Best mIoU Updated ************************